# Clustering with Foursquare API

In [2]:
print('Installing dependencies')
import sys
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} requests
!conda install --yes --prefix {sys.prefix} -c conda-forge geopy
!conda install --yes --prefix {sys.prefix} -c conda-forge folium=0.5.0

Installing dependencies
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /home/ivan/anaconda3/envs/Coursera_Capstone

  added / updated specs:
    - numpy


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> pkgs/main::ca-certificates-2020.1.1-0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2019.11.28-py37h~ --> pkgs/main::certifi-2019.11.28-py37_1
  openssl            conda-forge::openssl-1.1.1f-h516909a_0 --> pkgs/main::openssl-1.1.1f-h7b6447c_0


Preparing transaction: | done
Verifying transaction: - \ done
Executing transaction: / done
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [3]:
# library to handle requests
import requests 
# library for data analsysis
import pandas as pd 
# library to handle data in a vectorized manner
import numpy as np 
# library for random number generation
import random 

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# plotting library
import folium 

print('Libraries imported')


Libraries imported


In [93]:
CLIENT_ID = 'WSBDS3PHA2ZA2QRF1K2PFSPE1G2DOMXDFX5LTEJ2NCC5OUG1' # your Foursquare ID
CLIENT_SECRET = 'LOHFOAR0DHZK5WYJOU1N0FRMLVYOUKNYK3KBRCTT33YSEQBH' # your Foursquare Secret
VERSION = '20200403'
LIMIT = 1000
radius = 5000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: WSBDS3PHA2ZA2QRF1K2PFSPE1G2DOMXDFX5LTEJ2NCC5OUG1
CLIENT_SECRET:LOHFOAR0DHZK5WYJOU1N0FRMLVYOUKNYK3KBRCTT33YSEQBH


In [94]:
print('Ciudad de Mexico')
address = 'Azcapotzalco, Ciudad de Mexico'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Address')
print(address)
print('Coordinates')
print(latitude, longitude)
print('Radius')
print(radius)

Ciudad de Mexico
Address
Azcapotzalco, Ciudad de Mexico
Coordinates
19.4858148 -99.18420573027606
Radius
5000


In [95]:
# define URL
url_venues = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}'\
    .format(CLIENT_ID,
            CLIENT_SECRET,
            latitude,
            longitude,
            VERSION,
            radius)

# send GET request and get trending venues
results_venues = requests.get(url_venues).json()
print('Request code')
print(results_venues['meta']['code'])
print(results_venues)

Request code
200
{'meta': {'code': 200, 'requestId': '5e880edd71c428001b795f35'}, 'response': {'venues': [{'id': '4e2f0501d4c058fdbeef80e9', 'name': 'Jarritos Azcapotzalco', 'location': {'lat': 19.485444389764485, 'lng': -99.18402355999812, 'labeledLatLngs': [{'label': 'display', 'lat': 19.485444389764485, 'lng': -99.18402355999812}], 'distance': 45, 'cc': 'MX', 'city': 'Ciudad de México', 'state': 'Distrito Federal', 'country': 'México', 'formattedAddress': ['Ciudad de México, Distrito Federal', 'México']}, 'categories': [{'id': '4bf58dd8d48988d124941735', 'name': 'Office', 'pluralName': 'Offices', 'shortName': 'Office', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1585975243', 'hasPerk': False}, {'id': '4feb2778e4b086dbf28ee476', 'name': 'Embotelladora De Mexico (Red Cola)', 'location': {'address': 'Azcapotzalco', 'lat': 19.485869627632248, 'lng': -99.18374305582773, 'labeledLatLngs': [{'label': 

## Getting relevant part of JSON and transform it into a pandas dataframe

In [96]:
if len(results_venues['response']['venues']) == 0:
    venues_df = 'No trending venues are available at the moment!'

else:
    # assign relevant part of JSON to venues
    venues_json = results_venues['response']['venues']
    # Getting the name of the primary category
    for v in venues_json:
        v['categories'] = v['categories'][0]['name']
    # tranform venues into a dataframe
    venues_df = json_normalize(venues_json)
    ## Preprocessing 

print('Step completed')


Step completed


/home/ivan/anaconda3/envs/Coursera_Capstone/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


In [97]:
venues_df_clr = pd.DataFrame({
    'name': venues_df['name'],
    'categories': venues_df['categories'],
    'distance': venues_df['location.distance'],
    'zip_code': venues_df['location.postalCode'],
    'lat': venues_df['location.lat'],
    'lng': venues_df['location.lng']
})
venues_df_clr.head()


,name,categories,distance,zip_code,lat,lng
0,Jarritos Azcapotzalco,Office,45,NaN,19.485444,-99.184024
1,Embotelladora De Mexico (Red Cola),Factory,48,NaN,19.485870,-99.183743
2,Farmacias Similares F0652 Azcapotzalco,Pharmacy,176,02010,19.485031,-99.185672
3,Embotelladora Aga,Factory,30,NaN,19.485582,-99.184060
4,Delegación Azcapotzalco,Town Hall,810,02000,19.479504,-99.188060


In [98]:
venues_df_clr['zip_code'].unique()

array([nan, '02010', '02000', '02080'], dtype=object)

In [99]:
venues_df_clr.shape


(30, 6)